In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
import pandas as pd

In [2]:
ctlg_dir = '../../../data/catalogues'
gaia_path = f'{ctlg_dir}/gaia/gaia-coords.csv'
sdss_path = f'{ctlg_dir}/sdss/sdss-coords.csv'
save_path = f'{ctlg_dir}/gaia-sdss/cross-match.csv'

In [3]:
gaia_coords_df = pd.read_csv(gaia_path)
sdss_coords_df = pd.read_csv(sdss_path)

In [4]:
gaia_ra = gaia_coords_df['ra']
gaia_dec = gaia_coords_df['dec']
gaia_coords = SkyCoord(ra=gaia_ra*u.degree, dec=gaia_dec*u.degree)

sdss_ra = sdss_coords_df['PLUG_RA']
sdss_dec = sdss_coords_df['PLUG_DEC']
sdss_coords = SkyCoord(ra=sdss_ra*u.degree, dec=sdss_dec*u.degree)

In [5]:
# Find nearest neighbor

idx, d2d, _ = gaia_coords.match_to_catalog_sky(sdss_coords)
distances = d2d.arcsecond
keep = distances <= 1

In [6]:
# Filter elements in cross-match

gaia_cross = gaia_coords_df.iloc[keep]
sdss_cross = sdss_coords_df.iloc[idx][keep]
print(gaia_cross.shape[0], gaia_cross.index.unique().shape)
print(sdss_cross.shape[0], sdss_cross.index.unique().shape)

4607 (4607,)
4607 (4607,)


In [7]:
# Build cross-match dataset

gaia_cross = gaia_cross.reset_index(drop=True)
gaia_cross = gaia_cross.add_suffix('_gaia')
sdss_cross = sdss_cross.reset_index(drop=True)
sdss_cross = sdss_cross.add_suffix('_sdss')
cross_match = gaia_cross.join(sdss_cross)
cross_match.to_csv(save_path, index=False)